In [0]:
import datetime

import numpy as np
import pandas as pd
import os

import io

In [1]:
from google.colab import files

uploaded = files.upload()

Saving group_type.xlsx to group_type (1).xlsx


In [0]:
GTYPE_FILE = pd.ExcelFile('group_type (1).xlsx')

CertType_df = pd.read_excel(GTYPE_FILE, 'CertType')
BranchType_df = pd.read_excel(GTYPE_FILE, 'BranchType')

In [0]:
DATA_FILE = pd.ExcelFile('Data-Thailand-HR_masked.xlsx')
EDU = pd.read_excel(DATA_FILE, 'Education').iloc[:, :6]

In [57]:
EDU.set_index('Certificate Text', inplace=True)
EDU['CertTypeGroup'] = CertType_df.set_index('Certificate Text')['Certificate Text Type Group']
EDU['CertLevel'] = CertType_df.set_index('Certificate Text')['Level']
EDU.reset_index(inplace=True)

EDU.set_index('Branch of study 1 Text', inplace=True)
EDU['BranchTypeGroup'] = BranchType_df.set_index('Branch of study 1 Text')['Branch Type Group']
EDU.reset_index(inplace=True)

EDU.columns = EDU.columns.str.strip()
EDU.head()

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup
0,General Studies,Attended,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attend,NaN,Other
1,No Specific Branch,Attended,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attend,NaN,Other
2,No Specific Branch,Attended,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attend,NaN,Other
3,Education,Attended,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attend,NaN,EDU
4,Business-Economics,Bachelor,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,3.0,BUSINESS


In [0]:
attendLv_df = EDU[EDU['CertLevel'].isnull()] 
otherLv_df = EDU[EDU['CertLevel'].notnull()]

In [53]:
attendCountLv_df = attendLv_df.groupby('Serial Number').size().reset_index(name='AttendCount')
attendCountLv_df.head()

,Serial Number,AttendCount
0,1,1
1,2,1
2,3,1
3,4,1
4,6,1


In [52]:
otherMaxLv_df = otherLv_df.groupby('Serial Number')['CertLevel'].max().reset_index(name='MaxLevel')
otherMaxLv_df.head()

,Serial Number,MaxLevel
0,5,3.0
1,7,2.0
2,11,3.0
3,12,2.0
4,16,3.0


In [108]:
otherMaxLv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4305 entries, 0 to 4304
Data columns (total 2 columns):
Serial Number    4305 non-null int64
MaxLevel         4305 non-null float64
dtypes: float64(1), int64(1)
memory usage: 67.3 KB


In [104]:
cert_df = pd.merge(attendCountLv_df, otherMaxLv_df,  how='outer', left_on=['Serial Number'], right_on = ['Serial Number']).fillna(0).sort_values('Serial Number')
cert_df.head()

,Serial Number,AttendCount,MaxLevel
0,1,1.0,0.0
1,2,1.0,0.0
2,3,1.0,0.0
3,4,1.0,0.0
732,5,0.0,3.0


In [65]:
EDU.head()

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup
0,General Studies,Attended,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attend,NaN,Other
1,No Specific Branch,Attended,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attend,NaN,Other
2,No Specific Branch,Attended,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attend,NaN,Other
3,Education,Attended,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attend,NaN,EDU
4,Business-Economics,Bachelor,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,3.0,BUSINESS


In [66]:
EDU[EDU['Serial Number'] == 4839]

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup
7081,Languages,Bachelor,4839,2005-05-24,2009-03-08 00:00:00,SILPAKORN UNIVERSITY,Bachelor,3.0,Languages
7082,Languages,Attended No Degree,4839,2009-03-30,2010-02-26 00:00:00,KYUNGHEE UNIVERSITY,Attend,NaN,Languages
7083,Other Degree,Bachelor,4839,2010-02-02,2014-02-19 00:00:00,KYUNGHEE UNIVERSITY,Bachelor,3.0,Other
7084,Languages,Attended No Degree,4839,2013-01-07,2013-01-27 00:00:00,HELP UNIVERSITY,Attend,NaN,Languages
7085,Languages,Attended No Degree,4839,2016-02-27,2016-04-16 00:00:00,SOONGSIL UNIVERSITY,Attend,NaN,Languages


In [86]:
branchTypeGroup_df = pd.get_dummies(EDU['BranchTypeGroup'])
branchTypeGroup_df.head()

,BUSINESS,CHEM,COM,EDU,ENG_CHEM,ENG_CIVIL,ENG_COM/ELEC,ENG_ENV,ENG_IND,ENG_OTHER,ENG_PET/CHEM,Finance,Languages,Law,Other,Science,Social
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [95]:
branchType_df = pd.concat([EDU, branchTypeGroup_df], axis=1)
branchType_df.head()

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup,BUSINESS,...,ENG_ENV,ENG_IND,ENG_OTHER,ENG_PET/CHEM,Finance,Languages,Law,Other,Science,Social
0,General Studies,Attended,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
1,No Specific Branch,Attended,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
2,No Specific Branch,Attended,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
3,Education,Attended,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attend,NaN,EDU,0,...,0,0,0,0,0,0,0,0,0,0
4,Business-Economics,Bachelor,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,3.0,BUSINESS,1,...,0,0,0,0,0,0,0,0,0,0


In [96]:
branchType_df[branchType_df['Serial Number'] == 4839]

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup,BUSINESS,...,ENG_ENV,ENG_IND,ENG_OTHER,ENG_PET/CHEM,Finance,Languages,Law,Other,Science,Social
7081,Languages,Bachelor,4839,2005-05-24,2009-03-08 00:00:00,SILPAKORN UNIVERSITY,Bachelor,3.0,Languages,0,...,0,0,0,0,0,1,0,0,0,0
7082,Languages,Attended No Degree,4839,2009-03-30,2010-02-26 00:00:00,KYUNGHEE UNIVERSITY,Attend,NaN,Languages,0,...,0,0,0,0,0,1,0,0,0,0
7083,Other Degree,Bachelor,4839,2010-02-02,2014-02-19 00:00:00,KYUNGHEE UNIVERSITY,Bachelor,3.0,Other,0,...,0,0,0,0,0,0,0,1,0,0
7084,Languages,Attended No Degree,4839,2013-01-07,2013-01-27 00:00:00,HELP UNIVERSITY,Attend,NaN,Languages,0,...,0,0,0,0,0,1,0,0,0,0
7085,Languages,Attended No Degree,4839,2016-02-27,2016-04-16 00:00:00,SOONGSIL UNIVERSITY,Attend,NaN,Languages,0,...,0,0,0,0,0,1,0,0,0,0


In [97]:
branchType_list = EDU['BranchTypeGroup'].unique()
branchType_list

array([u'Other', u'EDU', u'BUSINESS', u'Finance', u'ENG_IND', u'Law',
       u'COM', u'Science', u'ENG_PET/CHEM', u'ENG_COM/ELEC', u'CHEM',
       u'ENG_CIVIL', u'Social', u'Languages', u'ENG_OTHER', u'ENG_CHEM',
       u'ENG_ENV'], dtype=object)

In [99]:
branchType_df = branchType_df.groupby(['Serial Number'])[branchType_list].max().reset_index()
branchType_df[branchType_df.index == 4839]

,Serial Number,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
4839,4883,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
branchType_df.head(10)

,Serial Number,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [102]:
cert_df.head(10)

,Serial Number,AttendCount,MaxLevel
0,1,1.0,0.0
1,2,1.0,0.0
2,3,1.0,0.0
3,4,1.0,0.0
4,6,1.0,0.0
5,8,2.0,0.0
6,9,1.0,0.0
7,10,1.0,0.0
8,11,1.0,3.0
9,13,1.0,0.0


In [107]:
all_df = pd.merge(cert_df, branchType_df,  how='outer', left_on=['Serial Number'], right_on = ['Serial Number'])
all_df.head()

,Serial Number,AttendCount,MaxLevel,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
0,1,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
